# 动态条形图的一种改进


### 1. 初始设置

首先，导入所需的库。

In [2]:
# 导入所需的库
import pandas as pd
import akshare as ak

### 2. 定义数据

In [4]:
# 当需要更新数据时，去掉注释后运行一次

# 从 akshare 获取历史数据，速度比较慢
df_all_history = ak.covid_19_history()

# 保存到 Excel 文件中，以便下次快速读取数据
df_all_history.to_excel('./data/covid_history.xlsx')

In [5]:
# 你可以前往 https://github.com/linjiwx/mp 下载画图用的数据和完整代码
df_all = pd.read_excel('./data/covid_history.xlsx', index_col=0).fillna('')

# 将字符串格式的日期转换为日期格式
df_all['date'] = pd.to_datetime(df_all['date'])

# 将时间格式转为字符串格式的日期，以 YYYY-mm-dd 的形式保存到另外一列
df_all['dates'] = df_all['date'].apply(lambda x:x.strftime('%Y-%m-%d'))

# 添加现存确诊列
df_all['current'] = df_all['confirmed'] - df_all['cured'] - df_all['dead']

# 排除省份的数据，只保留国家的数据，设置日期为索引
dfc = df_all.query("province=='' and country!='钻石公主号邮轮'").set_index('date')

# 用透视表整理数据格式
df_pivot = dfc.pivot_table(values='confirmed', index='country', columns='dates').fillna(0)

# 根据国家代码，插入国旗图片的链接
df_country = pd.read_excel('./data/国家代码.xlsx', index_col=0)
covid_pivot = df_country.join(df_pivot, how='right').fillna(0)
image_url = ['https://www.countryflags.io/'+str(i)+'/flat/64.png' for i in covid_pivot.代码]
covid_pivot.insert(1, '图片链接', image_url)

# 删除没用的列
covid_pivot = covid_pivot.drop(columns = ['代码'])

# 按列排序
covid_pivot = covid_pivot.sort_values(by=['2020-07-24'], ascending=False)

# 保存到 Excel 文件
covid_pivot.to_excel('./data/covid_pivot.xlsx')

### 3. 开始作图

借助第三方工具，分三步完成：

（1）注册并登录 https://app.flourish.studio/ 

（2）在浏览器中访问 https://public.flourish.studio/visualisation/3280860/

（3）点击右上角的 「Duplicate and edit」，然后就可以自己随意调整数据和显示效果。